<a href="https://colab.research.google.com/github/poandpo/project3/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    !pip install eli5
    !pip install catboost

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
import pandas as pd
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## 데이터 탐색

In [ ]:
df.shape, test.shape

((103904, 25), (25976, 25))

In [ ]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


In [ ]:
test.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,4,3,4,3,5,5,5,5,2,5,5,50,44.0,satisfied
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,1,5,4,5,4,4,4,4,3,4,5,0,0.0,satisfied
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,4,2,2,2,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,2,3,4,4,1,1,1,1,3,1,4,0,6.0,satisfied
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,3,4,1,2,2,2,2,2,4,2,4,0,20.0,satisfied


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [ ]:
df.T.duplicated(subset=None, keep='first')

Unnamed: 0                           False
id                                   False
Gender                               False
Customer Type                        False
Age                                  False
Type of Travel                       False
Class                                False
Flight Distance                      False
Inflight wifi service                False
Departure/Arrival time convenient    False
Ease of Online booking               False
Gate location                        False
Food and drink                       False
Online boarding                      False
Seat comfort                         False
Inflight entertainment               False
On-board service                     False
Leg room service                     False
Baggage handling                     False
Checkin service                      False
Inflight service                     False
Cleanliness                          False
Departure Delay in Minutes           False
Arrival Del

In [ ]:
[(x, df[x].isnull().sum()) for x in df.columns if df[x].isnull().any()]

[('Arrival Delay in Minutes', 310)]

In [ ]:
df.head(5)

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report")

In [ ]:
df['Class'].unique()

array(['Eco Plus', 'Business', 'Eco'], dtype=object)

In [ ]:
columns_name = []
for x in df.columns:
  columns_name.append(x)

print(columns_name)    

['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class', 'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes', 'satisfaction']


In [ ]:
df['satisfaction'].value_counts(normalize=True)

neutral or dissatisfied    0.566667
satisfied                  0.433333
Name: satisfaction, dtype: float64

In [ ]:
import seaborn as sns

sns.countplot(df['satisfaction'],)

## 데이터 전처리

In [ ]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


In [ ]:
def clean_df(df):
    # 지연된 시간 합치기
    df['Delay_Minutes']=df['Departure_Delay_in_Minutes'] + df['Arrival_Delay_in_Minutes']
    df.drop(labels=['Unnamed: 0','id','Departure_Delay_in_Minutes','Arrival_Delay_in_Minutes'], axis=1,inplace=True )

    # 결측치 대체
    df['Delay_Minutes']= df['Delay_Minutes'].fillna(0)
    
    #인코더
    df["Gender"] = df["Gender"].map({'Male': 0,'Female': 1})
    df['Customer_Type']=df['Customer_Type'].map({'Loyal Customer': 0,'disloyal Customer': 1})
    df['Type_of_Travel']=df['Type_of_Travel'].map({'Personal Travel': 0,'Business travel': 1})
    df['Class']=df['Class'].map({'Eco': 0,'Eco Plus': 1, 'Business':2 })
    df['satisfaction']=df['satisfaction'].map({'neutral or dissatisfied': 0,'satisfied': 1})

    # 컬럼명 변경
    df.rename(columns={'Type_of_Travel':'Travel_Tyep', 'Flight_Distance':'Distance','Inflight_wifi_service':'wifi_service', 'Departure_Arrival_time_convenient':'time_convenient','Ease_of_Online_booking':'Online_booking', 'Food_and_drink': 'Food_drink',
      
                      'Inflight_entertainment':'entertainment'},inplace=True )  
clean_df(df)
clean_df(test)  

In [ ]:
[(x, df[x].isnull().sum()) for x in df.columns if df[x].isnull().any()]

[('Arrival Delay in Minutes', 310)]

In [ ]:
[(x, test[x].isnull().sum()) for x in test.columns if test[x].isnull().any()]

[('Arrival Delay in Minutes', 83)]

In [ ]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


In [ ]:
from sklearn.model_selection import train_test_split
X=df[['Gender','Travel_Tyep','Customer_Type','Class','Age','wifi_service', 'Online_booking','entertainment','Checkin_service','Inflight_service', 'time_convenient']] #'price'를 제외한 df를 특성으로 지정
y=df['satisfaction']
X_train, X_val, y_train, y_val = train_test_split(X,y,
                                                    test_size=0.3, 
                                                    random_state=2)

X_test = test[['Gender','Travel_Tyep','Customer_Type','Class','Age','wifi_service', 'Online_booking','entertainment','Checkin_service','Inflight_service', 'time_convenient']]
y_test = test['satisfaction']

In [ ]:
X_train.shape, X_val.shape, X_test.shape

In [ ]:
X_train.head()

## 훈련

### LGBMClassifier

In [ ]:
from lightgbm import  LGBMClassifier

model =LGBMClassifier(colsample_bytree= 0.85, 
                        max_depth= 15, 
                        min_split_gain= 0.1, 
                        n_estimators= 200, 
                        num_leaves= 50, 
                        reg_alpha= 1.2, 
                        reg_lambda= 1.2, 
                        subsample= 0.95, 
                        subsample_freq= 20)

eval_set = [(X_train, y_train), 
            (X_val, y_val)]

model.fit(X_train, y_train, 
          eval_set=eval_set,
          #eval_metric=error,
          early_stopping_rounds=50
         )     

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred)) 

In [ ]:
y_pred2 = model.predict(X_val)
print(classification_report(y_val, y_pred2)) 

### XGB

In [ ]:
from xgboost import XGBClassifier

model2 = XGBClassifier(
    n_estimators=1000,  # <= 1000 트리로 설정했지만, early stopping 에 따라 조절됩니다.
    max_depth=7,        # default=3, high cardinality 특성을 위해 기본보다 높여 보았습니다.
    learning_rate=0.2,
#     scale_pos_weight=ratio, # imbalance 데이터 일 경우 비율을 적용합니다.
    n_jobs=-1
)

eval_set = [(X_train, y_train), 
            (X_val, y_val)]

model2.fit(X_train, y_train, 
          eval_set=eval_set,
          eval_metric='error', # #(wrong cases)/#(all cases)
          early_stopping_rounds=50
         ) 

In [ ]:
y_pred_1 = model.predict(X_train)
print(classification_report(y_train, y_pred_1)) 

In [ ]:
y_pred_2= model.predict(X_val)
print(classification_report(y_val, y_pred_2)) 

In [ ]:
!pip install catboost

### Cat

In [ ]:
from catboost import CatBoostClassifier

model3=CatBoostClassifier(n_estimators=1000,
                       learning_rate= 0.15513836571095696, 
                       max_depth= 10,
                       random_state=10)   
    
eval_set = [(X_train, y_train), 
            (X_val, y_val)]

model3.fit(X_train, y_train, 
          eval_set=eval_set,
          #eval_metric='error', # #(wrong cases)/#(all cases)
          early_stopping_rounds=50
         )           


0:	learn: 0.4361243	test: 0.4361243	test1: 0.4358540	best: 0.4358540 (0)	total: 70.5ms	remaining: 1m 10s
1:	learn: 0.3875520	test: 0.3875520	test1: 0.3872079	best: 0.3872079 (1)	total: 91.6ms	remaining: 45.7s
2:	learn: 0.3490146	test: 0.3490146	test1: 0.3483887	best: 0.3483887 (2)	total: 110ms	remaining: 36.6s
3:	learn: 0.3178734	test: 0.3178734	test1: 0.3170430	best: 0.3170430 (3)	total: 130ms	remaining: 32.3s
4:	learn: 0.2930809	test: 0.2930809	test1: 0.2922526	best: 0.2922526 (4)	total: 150ms	remaining: 29.8s
5:	learn: 0.2736785	test: 0.2736785	test1: 0.2728325	best: 0.2728325 (5)	total: 169ms	remaining: 28s
6:	learn: 0.2589725	test: 0.2589725	test1: 0.2581324	best: 0.2581324 (6)	total: 188ms	remaining: 26.7s
7:	learn: 0.2473977	test: 0.2473977	test1: 0.2465365	best: 0.2465365 (7)	total: 208ms	remaining: 25.7s
8:	learn: 0.2369741	test: 0.2369741	test1: 0.2360858	best: 0.2360858 (8)	total: 227ms	remaining: 25s
9:	learn: 0.2299139	test: 0.2299139	test1: 0.2290716	best: 0.2290716 (9)	t

In [ ]:
y_pred_3 = model.predict(X_train)
print(classification_report(y_train, y_pred_3)) 

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     41211
           1       0.96      0.93      0.95     31521

    accuracy                           0.95     72732
   macro avg       0.96      0.95      0.95     72732
weighted avg       0.95      0.95      0.95     72732



In [ ]:
y_pred_4= model.predict(X_val)
print(classification_report(y_val, y_pred_4)) 

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     17668
           1       0.96      0.94      0.95     13504

    accuracy                           0.95     31172
   macro avg       0.95      0.95      0.95     31172
weighted avg       0.95      0.95      0.95     31172



### 평가 지료 비교

Light
           
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     17668
           1       0.96      0.94      0.95     13504

    accuracy                           0.95     31172




XGB

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     17668
           1       0.96      0.94      0.95     13504

    accuracy                           0.95     31172


CAT

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     17668
           1       0.96      0.94      0.95     13504

    accuracy                           0.95     31172



**평가지표를 비교하니 대부분 비슷하므로 속도가 빠른 LightGBM로 결정**




In [ ]:
y_pred3 = model.predict(X_test)
print(classification_report(y_test, y_pred3)) 

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     14573
           1       0.95      0.94      0.95     11403

    accuracy                           0.95     25976
   macro avg       0.95      0.95      0.95     25976
weighted avg       0.95      0.95      0.95     25976



### cf ) 순열 중요도

In [ ]:
pip install eli5

     |████████████████████████████████| 106 kB 28.5 MB/s 


In [ ]:
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance

# permuter 정의
permuter = PermutationImportance(
    model, # model
    scoring='accuracy', # metric
    n_iter=5, # 다른 random seed를 사용하여 5번 반복
    random_state=2
)

# permuter 계산은 preprocessing 된 X_val을 사용합니다.
#X_val_transformed = pipe.named_steps['preprocessing'].transform(X_val)

# 실제로 fit 의미보다는 스코어를 다시 계산하는 작업입니다
permuter.fit(X_val, y_val);

In [ ]:
feature_names = X_val.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None, # top n 지정 가능, None 일 경우 모든 특성 
    feature_names=feature_names # list 형식으로 넣어야 합니다
)

Weight,Feature
0.1927 ± 0.0044,wifi_service
0.1507 ± 0.0028,Travel_Tyep
0.0807 ± 0.0012,Customer_Type
0.0432 ± 0.0016,Class
0.0384 ± 0.0021,Inflight_service
0.0351 ± 0.0019,entertainment
0.0321 ± 0.0007,Checkin_service
0.0143 ± 0.0014,time_convenient
0.0087 ± 0.0010,Age
0.0065 ± 0.0012,Online_booking


## 모델 저장하기

In [ ]:
import pickle
from sklearn.externals import joblib


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
pickle.dump(model, open('light.pkl', 'wb'))

In [ ]:
saved_model = pickle.dumps(model)

clf_from_pickle = pickle.loads(saved_model)
clf_from_pickle.predict(X)

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
joblib.dump(model, 'light.pkl') 

['light.pkl']

In [ ]:
clf_from_joblib = joblib.load('light.pkl') 
clf_from_joblib.predict(X)


array([0, 0, 1, ..., 0, 0, 0])